In [ ]:
import pandas as pd
from ortools.linear_solver import pywraplp as OR

In [ ]:
items = pd.read_csv('knapsack_example.csv', index_col=0)

In [ ]:
def knapsack(items, capacity, integer = False):
    """Model for solving the Knapsack problem.
    
    Args:
        items (pd.DataFrame): A table indexd by items with a column for value and weight
        capcity (int): An integer-capacity for the knapsack
        integer (bool): True if the variables should be integer. False otherwise.
    """
    ITEMS = list(items.index)        # set of items
    v = items.to_dict()['value']     # value for each item 
    w = items.to_dict()['weight']    # weight for each item
    W = capacity                     # capacity of the knapsack
    
    # define model
    m = OR.Solver('knapsack', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    # decision variables
    x = {}    
    for i in ITEMS:
        if integer:
            x[i] = m.IntVar(0, 1, 'x_%d' % (i)) 
        else:
            x[i] = m.NumVar(0, 1, 'x_%d' % (i)) 
        
    # define objective function here
    m.Maximize(sum(v[i]*x[i] for i in ITEMS))
    
    # weight must not exceed capacity
    m.Add(sum(w[i]*x[i] for i in ITEMS) <= W)
    
    return m, x

In [ ]:
def solve(m):
    m.Solve()
    print('Solution:')
    print('Objective value =', m.Objective().Value())
    for var in m.variables():
        print(var.name(), ':',  var.solution_value())

In [ ]:
m, x = knapsack(items, 18)
solve(m)

In [ ]:
m, x = knapsack(items, 18, integer=True)
solve(m)